In [55]:
import pandas as pd
import numpy as np

In [56]:
import collections

In [66]:
import random

In [69]:
import tensorflow as tf

In [13]:
data = pd.read_csv('./new_train3.csv')

In [14]:
data.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,07-01 00,A,False,"[[-861 4114],[-862 4114],[-863 4114],[-862 411..."
1,1372637303620000596,B,NaN,Av. Boavista,20000596,07-01 00,A,False,"[[-863 4115],[-864 4115],[-864 4116],[-865 411..."
2,1372636951620000320,C,NaN,NaN,20000320,07-01 00,A,False,"[[-861 4114],[-862 4114],[-863 4114],[-862 411..."
3,1372636854620000520,C,NaN,NaN,20000520,07-01 00,A,False,"[[-857 4115],[-857 4114],[-858 4114],[-861 411..."
4,1372637091620000337,C,NaN,NaN,20000337,07-01 00,A,False,"[[-864 4118],[-864 4117],[-865 4117],[-866 411..."


In [46]:
data = data.drop(['MISSING_DATA', 'TRIP_ID', 'DAY_TYPE'], 1)

In [47]:
data.head()

,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,POLYLINE
0,C,NaN,NaN,20000589,07-01 00,"[[-861 4114],[-862 4114],[-863 4114],[-862 411..."
1,B,NaN,Av. Boavista,20000596,07-01 00,"[[-863 4115],[-864 4115],[-864 4116],[-865 411..."
2,C,NaN,NaN,20000320,07-01 00,"[[-861 4114],[-862 4114],[-863 4114],[-862 411..."
3,C,NaN,NaN,20000520,07-01 00,"[[-857 4115],[-857 4114],[-858 4114],[-861 411..."
4,C,NaN,NaN,20000337,07-01 00,"[[-864 4118],[-864 4117],[-865 4117],[-866 411..."


In [48]:
words = list()
pos_list = list()
cnt = 0
for i in range(data.shape[0]):
    pos_list.append(cnt)
    for name in data.columns:
        if name != 'POLYLINE':
            words.append(str(data[name][i]))
            cnt = cnt + 1
        else:
            #for li in data[name][i]:
            tmpstr = data[name][i]
            tmpstr = tmpstr[1:len(tmpstr)-1]
            tmpstr = tmpstr.split(",")
            for tmpitem in tmpstr:
                words.append(tmpitem)
            cnt = cnt + 1

len(words)

22212788

In [57]:
vocabulary_size = 5000

def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count = unk_count + 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
print(dictionary['A'])
print(dictionary['B'])
print(dictionary['C'])

Most common words (+UNK) [['UNK', 1120764], ('nan', 2249991), ('B', 817881), ('[-861 4114]', 751207), ('C', 528019)]
Sample data [4, 1, 1, 329, 0, 3, 11, 28, 5, 9]
13
2
4


In [63]:
start_index = 0

def generate_cbow_batch(batch_size, context_size):
    batch_size = batch_size * context_size
    global start_index
  
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size // context_size, 1), dtype=np.int32)

    for i in range(batch_size // context_size):
        labels[i, 0] = data[pos_list[i+1]-1]
        for j in range(context_size):
            batch[i*context_size + j] = data[pos_list[i]+j]
            
    return batch, labels

b_size = 24
c_size = 6
batch, labels = generate_cbow_batch(batch_size=b_size, context_size=c_size)

for i in range(b_size // (c_size)):
    print("CONTEXT FOR %s:" % reverse_dictionary[labels[i,0]])
    for j in range(c_size):
        print(reverse_dictionary[batch[i*(c_size) + j]])
    print

CONTEXT FOR [-861 4114]:
C
nan
nan
20000589
UNK
[-861 4114]
CONTEXT FOR nan:
[-862 4114]
[-863 4114]
[-862 4115]
[-863 4115]
B
nan
CONTEXT FOR [-864 4116]:
Av. Boavista
20000596
UNK
[-863 4115]
[-864 4115]
[-864 4116]
CONTEXT FOR C:
[-865 4116]
[-866 4116]
[-867 4116]
[-866 4116]
[-866 4117]
C


In [70]:
### initialization of the graph ###

batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
context_window = 6 # How many words to consider left and right
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 64 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size * context_window])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
    # Variables.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    segments = tf.constant([x / context_window for x in range(batch_size * context_window)])
    softmax_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    compressed_embeddings = tf.segment_sum(embed, segments) # merging couple of embeded words into one input
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases, compressed_embeddings,
                                   train_labels, num_sampled, vocabulary_size))

    # Optimizer.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))
    
    input = [dictionary['C'], dictionary['nan'], dictionary['Av. Boavista'], dictionary['20000320'],
             dictionary['07-01 11'], dictionary['[-861 4115]']] * batch_size
    logits = tf.matmul(compressed_embeddings, tf.transpose(softmax_weights)) + softmax_biases
    y_ = tf.nn.softmax(logits)

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [ ]:
num_steps = 7000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    average_loss = 0
    for step in range(num_steps):
        batch_data, batch_labels = generate_cbow_batch(
        batch_size, context_window)
        feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += l
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
        # note that this is expensive (~20% slowdown if computed every 500 steps)
    
    final_embeddings = normalized_embeddings.eval()
    # feeds the algorithm a context and gets the most probable autocompletion/autocorrection
    feed_dict = {train_dataset : input}
    _, y = session.run([logits, y_], feed_dict=feed_dict)
 
    top_k = 4 # number of nearest neighbors
    for i in range(1):
        print [reverse_dictionary[x] for x in (-y[i, :]).argsort()[1:top_k+1]]